<a href="https://colab.research.google.com/github/w-dan/ML-practica1/blob/main/practica1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
import numpy as np
import pandas as pd

# preprocesamiento de datos continuos
import sklearn.preprocessing as preprocessing
from sklearn.preprocessing import MinMaxScaler

# preprocesamiento de datos discretos
from sklearn.preprocessing import OrdinalEncoder

# preprocesamiento de texto
from sklearn.feature_extraction.text import CountVectorizer

# regresion y metricas
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error
import matplotlib.pyplot as plt

In [33]:
train_url = "https://gist.githubusercontent.com/w-dan/77e46a090ae0c2fec5bb2ecdf6e09cc8/raw/779897c43ee5976e6fe8de37cd1d6575f6c2a2ad/practica1-train.csv"
test_url = "https://gist.githubusercontent.com/w-dan/77e46a090ae0c2fec5bb2ecdf6e09cc8/raw/779897c43ee5976e6fe8de37cd1d6575f6c2a2ad/practica1-test.csv"

df_train = pd.read_csv(train_url)
df_test  = pd.read_csv(test_url)

In [34]:
# eliminar la variable id(no aporta nada) y separar price(variable objetivo)
y = df_train['Price']
df_train = df_train.drop('LaptopId', axis=1)
df_train = df_train.drop('Price', axis=1)

In [35]:
# primer intento de encodear valores discretos
enc = OrdinalEncoder()
df_train[['Product', 'TypeName', 'Company']] = enc.fit_transform(df_train[['Product', 'TypeName', 'Company']])
df_train

,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight
0,16.0,262.0,3.0,13.3,Full HD 1920x1080,Intel Core i5 6200U 2.3GHz,4GB,128GB SSD,Intel HD Graphics 520,Windows 7,1.2kg
1,7.0,326.0,4.0,13.3,IPS Panel Full HD / Touchscreen 1920x1080,Intel Core i7 7500U 2.7GHz,8GB,256GB SSD,Intel HD Graphics 620,Windows 10,1.11kg
2,4.0,191.0,3.0,15.6,Full HD 1920x1080,Intel Core i7 8550U 1.8GHz,8GB,128GB SSD + 1TB HDD,Intel UHD Graphics 620,Windows 10,2.02kg
3,7.0,95.0,4.0,13.3,IPS Panel Full HD 1920x1080,Intel Core i7 7500U 2.7GHz,8GB,256GB SSD,Nvidia GeForce MX150,Windows 10,1.38kg
4,4.0,215.0,4.0,12.5,Full HD / Touchscreen 1920x1080,Intel Core i5 7200U 2.5GHz,8GB,256GB SSD,Intel HD Graphics,Windows 10,1.36kg
...,...,...,...,...,...,...,...,...,...,...,...
907,4.0,187.0,0.0,13.3,Full HD / Touchscreen 1920x1080,Intel Core i5 7200U 2.5GHz,8GB,256GB SSD,Intel HD Graphics 620,Windows 10,1.68kg
908,2.0,103.0,1.0,17.3,Full HD 1920x1080,Intel Core i7 7700HQ 2.8GHz,8GB,128GB SSD + 1TB HDD,Nvidia GeForce GTX 1050,Windows 10,3kg
909,4.0,189.0,3.0,15.6,Full HD 1920x1080,Intel Core i7 7500U 2.7GHz,16GB,2TB HDD,AMD Radeon R7 M445,Windows 10,2.32kg
910,4.0,183.0,3.0,15.6,Full HD 1920x1080,Intel Core i7 7500U 2.7GHz,8GB,1TB HDD,AMD Radeon R5 M430,Linux,2.2kg


In [36]:
# obtener el valor numérico de la RAM
df_train['Ram'] = df_train['Ram'].map(lambda x: str(x)[:-2])
df_train

,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight
0,16.0,262.0,3.0,13.3,Full HD 1920x1080,Intel Core i5 6200U 2.3GHz,4,128GB SSD,Intel HD Graphics 520,Windows 7,1.2kg
1,7.0,326.0,4.0,13.3,IPS Panel Full HD / Touchscreen 1920x1080,Intel Core i7 7500U 2.7GHz,8,256GB SSD,Intel HD Graphics 620,Windows 10,1.11kg
2,4.0,191.0,3.0,15.6,Full HD 1920x1080,Intel Core i7 8550U 1.8GHz,8,128GB SSD + 1TB HDD,Intel UHD Graphics 620,Windows 10,2.02kg
3,7.0,95.0,4.0,13.3,IPS Panel Full HD 1920x1080,Intel Core i7 7500U 2.7GHz,8,256GB SSD,Nvidia GeForce MX150,Windows 10,1.38kg
4,4.0,215.0,4.0,12.5,Full HD / Touchscreen 1920x1080,Intel Core i5 7200U 2.5GHz,8,256GB SSD,Intel HD Graphics,Windows 10,1.36kg
...,...,...,...,...,...,...,...,...,...,...,...
907,4.0,187.0,0.0,13.3,Full HD / Touchscreen 1920x1080,Intel Core i5 7200U 2.5GHz,8,256GB SSD,Intel HD Graphics 620,Windows 10,1.68kg
908,2.0,103.0,1.0,17.3,Full HD 1920x1080,Intel Core i7 7700HQ 2.8GHz,8,128GB SSD + 1TB HDD,Nvidia GeForce GTX 1050,Windows 10,3kg
909,4.0,189.0,3.0,15.6,Full HD 1920x1080,Intel Core i7 7500U 2.7GHz,16,2TB HDD,AMD Radeon R7 M445,Windows 10,2.32kg
910,4.0,183.0,3.0,15.6,Full HD 1920x1080,Intel Core i7 7500U 2.7GHz,8,1TB HDD,AMD Radeon R5 M430,Linux,2.2kg


In [37]:
# por el momento voy a hacer pruebas con columnas que no creo que requieran mucho mas procesamiento
# de momento voy a usar (normalizar) ['Company', 'Product', 'TypeName', 'Inches', 'Weight', 'Ram']
y

0      1195.00
1      1349.00
2       855.00
3      1119.00
4      1472.20
        ...   
907     889.00
908    1168.00
909     989.99
910     599.90
911     859.00
Name: Price, Length: 912, dtype: float64

In [38]:
X = df_train[['Company', 'Product', 'TypeName', 'Inches', 'Ram']]
reg = LinearRegression()
reg.fit(X, y)

LinearRegression()

In [39]:
reg.coef_

array([ 15.52122085,   0.48442741,  23.89285625, -43.12689847,
       110.68303284])

In [40]:
X
# vale parece bastante mas razonable que antes
# ahora a normalizar

,Company,Product,TypeName,Inches,Ram
0,16.0,262.0,3.0,13.3,4
1,7.0,326.0,4.0,13.3,8
2,4.0,191.0,3.0,15.6,8
3,7.0,95.0,4.0,13.3,8
4,4.0,215.0,4.0,12.5,8
...,...,...,...,...,...
907,4.0,187.0,0.0,13.3,8
908,2.0,103.0,1.0,17.3,8
909,4.0,189.0,3.0,15.6,16
910,4.0,183.0,3.0,15.6,8


In [41]:
df_train[['Company', 'Product', 'TypeName', 'Inches', 'Ram']] = MinMaxScaler().fit_transform(df_train[['Company', 'Product', 'TypeName', 'Inches', 'Ram']])
X = df_train[['Company', 'Product', 'TypeName', 'Inches', 'Ram']]            # 'Product' es mucho menos relevante que las demas
X                                                                            # dataframe normalizado [0, 1]

,Company,Product,TypeName,Inches,Ram
0,0.888889,0.545833,0.6,0.444444,0.066667
1,0.388889,0.679167,0.8,0.444444,0.200000
2,0.222222,0.397917,0.6,0.763889,0.200000
3,0.388889,0.197917,0.8,0.444444,0.200000
4,0.222222,0.447917,0.8,0.333333,0.200000
...,...,...,...,...,...
907,0.222222,0.389583,0.0,0.444444,0.200000
908,0.111111,0.214583,0.2,1.000000,0.200000
909,0.222222,0.393750,0.6,0.763889,0.466667
910,0.222222,0.381250,0.6,0.763889,0.200000


In [42]:
X = df_train[['Company', 'Product', 'TypeName', 'Inches', 'Ram']]
reg.fit(X, y)


LinearRegression()

In [43]:
reg.predict(X)
reg.coef_           # parece que product es mucho mas relevante tras normalizar, por que?

array([ 279.38197522,  232.52515583,  119.46428125, -310.51366899,
       3320.49098521])

In [44]:
# no se que cojones estoy haciendo